# Momentum distributions

__Author:__ A. J. Tropiano [atropiano@anl.gov]<br/>
__Date:__ May 30, 2023

Plots single-nucleon momentum distributions which are calculated using SRG-evolved momentum projection operators with low-resolution nuclear wavefunctions approximated as a single Slater determinant of Woods-Saxon single-particle orbitals.

_Last update:_ January 5, 2024

In [1]:
# Python imports
from matplotlib.offsetbox import AnchoredText
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
# Imports from scripts
from scripts.figures import (
    label_kvnn, label_lambda, label_nucleus, line_styles, set_rc_parameters
)
from scripts.integration import momentum_mesh
from scripts.momentum_distributions import MomentumDistribution
from scripts.tools import convert_number_to_string, replace_periods

from test_momentum_distribution_script_v9 import load_momentum_distribution
from test_momentum_distribution_script_v9 import compute_normalization

### Set-up

In [3]:
def label_potential_lambda(kvnn, lamb):
    """Label for potential and SRG \lambda."""
    
    kvnn_label = label_kvnn(kvnn)
    lamb_label = rf"$({convert_number_to_string(lamb)})$"
    
    return kvnn_label + lamb_label

In [4]:
# Run this cell to turn on customized matplotlib graphics
set_rc_parameters()

In [5]:
# Save figures in the following directory
figures_directory = '../figures/momentum_distributions/'
vmc_directory = '../data/vmc/momentum_distributions/'

## Comparison to VMC

In [ ]:
def OLD_single_nucleon_momentum_distributions_multipanel(
        nuclei, nucleon, kvnn, lamb, x_limits=(0.0,6.0), y_limits=(1e-4,1e3),
        y_scale='log', show_vmc=False, save=False
):
    """Plot a single-nucleon momentum distribution with several built-in
    options.
    """
    
    # Initialize figure
    plt.close('all')
    row_number, col_number = 1, len(nuclei)
    f, axs = plt.subplots(row_number, col_number, sharex=True, sharey=True,
                          figsize=(4*col_number, 4*row_number))
        
    # Loop over nuclei
    for i, nucleus in enumerate(nuclei):
        
        nucleus_name, Z, N = nucleus
        if nucleon == 'proton':
            factor = Z
        elif nucleon == 'neutron':
            factor = N
            
        if y_scale == 'log':
            axs[i].set_yscale('log')
        else:
            axs[i].set_yscale('linear')
        
        # Compare to VMC calculation?
        if show_vmc:
            
            try:
                file_name = f"{nucleus_name}_single_nucleon_av18_ux.txt"
            except 
            data = np.loadtxt(vmc_directory + file_name)
            q_array_vmc = data[:,0]
            n_array_vmc = data[:,1] / factor
            n_errors_vmc = data[:,2] / factor
        
            if y_scale == 'linear':
            
                axs[i].errorbar(
                    q_array_vmc, n_array_vmc * q_array_vmc ** 2 ,
                    yerr=n_errors_vmc, color='black', label='VMC', linestyle='',
                    marker='.', zorder=3
                )
            
            else:
            
                axs[i].errorbar(
                    q_array_vmc, n_array_vmc, yerr=n_errors_vmc, color='black',
                    label='VMC', linestyle='', marker='.', zorder=3
                )

        # Load data from file
        q_array, q_weights, n_array, n_errors, n_ipm_array, _, _, _, _ = (
            load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb)
        )
    
        # Multiply by factor (2\pi)^3 / Z (N) to compare to VMC data
        n_array *= (2*np.pi) ** 3 / factor
        n_errors *= (2*np.pi) ** 3 / factor
        n_ipm_array *= (2*np.pi) ** 3 / factor

        # Add to plot
        if y_scale == 'linear':
        
            axs[i].plot(
                q_array, n_array * q_array ** 2,
                label='SRG', linewidth=2.0, zorder=2
            )
            
            axs[i].plot(q_array, n_ipm_array * q_array ** 2, label='IPM',
                        linestyle='dotted', linewidth=2.0, zorder=1)

        else:
        
            axs[i].plot(q_array, n_array, label='SRG', linewidth=2.0, zorder=2)
            
            axs[i].plot(q_array, n_ipm_array, label='IPM', linestyle='dotted',
                        linewidth=2.0, zorder=1)

        # Set axes limits
        axs[i].set_xlim(x_limits)
        axs[i].set_ylim(y_limits)

        # Set axes labels
        axs[i].set_xlabel(r"$q$ [fm" + r'$^{-1}$' + ']', fontsize=16)
        if i == 0:  # First column only
            if y_scale == 'linear' and nucleon == 'proton':
                ylabel = r"$q^2 n_{p}(q)/Z$ [fm]"
            elif y_scale == 'linear' and nucleon == 'neutron':
                ylabel = r"$q^2 n_{n}(q)/N$ [fm]"
            elif y_scale == 'log' and nucleon == 'proton':
                ylabel = r"$n_{p}(q)/Z$ [fm" + r'$^3$' + ']'
            elif y_scale == 'log' and nucleon == 'neutron':
                ylabel = r"$n_{n}(q)/N$ [fm" + r'$^3$' + ']'
            axs[i].set_ylabel(ylabel, fontsize=16)

        # Add nucleus label
        if y_scale == 'log':
            nucleus_label_loc = 'lower left'
        elif y_scale == 'linear':
            nucleus_label_loc = 'upper left'
        axs[i].add_artist(
            AnchoredText(label_nucleus(nucleus_name), loc=nucleus_label_loc,
                         prop=dict(size=18), frameon=False)
        )
        
    # Set legend in first panel only
    axs[0].legend(loc='upper right', fontsize=16, frameon=False)
    
    # Add potential and \lambda label in last panel
    axs[-1].add_artist(
        AnchoredText(label_potential_lambda(kvnn, lamb), loc='upper right',
                     prop=dict(size=18), frameon=False)
    )

    # Amount of white space in-between sub-plots
    f.subplots_adjust(wspace=0.1)

    # Save?
    if save:   
        file_name = replace_periods(
            f"{nucleon}_momentum_distributions_kvnn_{kvnn}_lamb_{lamb}"
            f"_{y_scale}_scale"
        )
        f.savefig(figures_directory + file_name + '.png')

In [ ]:
# TODO: Update labels to say CVMC where appropriate?
def single_nucleon_momentum_distributions_multipanel(
        nuclei, nucleon, kvnn, lamb, x_limits=(0.0,6.0), y_limits=(1e-4,1e3),
        y_scale='log', show_vmc=False, save=False
):
    """Plot a single-nucleon momentum distribution with several built-in
    options.
    """
    
    # Initialize figure
    plt.close('all')
    row_number, col_number = 1, len(nuclei)
    f, axs = plt.subplots(row_number, col_number, sharex=True, sharey=True,
                          figsize=(4*col_number, 4*row_number))
        
    # Loop over nuclei
    for i, nucleus in enumerate(nuclei):
        
        nucleus_name, Z, N = nucleus
        if nucleon == 'proton':
            factor = Z
        elif nucleon == 'neutron':
            factor = N
            
        if y_scale == 'log':
            axs[i].set_yscale('log')
        else:
            axs[i].set_yscale('linear')
        
        # Compare to VMC calculation?
        if show_vmc:
        
            # Two-body interaction only: AV18
            if nucleus_name != 'C12':
                file_name_2b = f"{nucleus_name}_single_nucleon_av18.txt"
                data_2b = np.loadtxt(vmc_directory + file_name_2b)
                q_vmc_2b = data_2b[:,0]
                n_vmc_2b = data_2b[:,1] / factor
                n_errors_2b = data_2b[:,2] / factor
            
            # Two-body and three-body interactions: AV18 + UIX
            file_name_3b = f"{nucleus_name}_single_nucleon_av18_uix.txt"
            data_3b = np.loadtxt(vmc_directory + file_name_3b)
            q_vmc_3b = data_3b[:,0]
            n_vmc_3b = data_3b[:,1] / factor
            n_errors_3b = data_3b[:,2] / factor
        
            if y_scale == 'linear':
            
                # AV18
                if nucleus_name != 'C12':
                    axs[i].errorbar(
                        q_vmc_2b, n_vmc_2b * q_vmc_2b ** 2,
                        yerr=n_errors_2b * q_vmc_2b ** 2, color='tab:red',
                        label='VMC 2b', linestyle='', marker='s', markersize=4,
                        zorder=1
                    )
                # AV18 + UIX
                axs[i].errorbar(
                    q_vmc_3b, n_vmc_3b * q_vmc_3b ** 2,
                    yerr=n_errors_3b * q_vmc_3b ** 2, color='tab:blue',
                    label='VMC 2b+3b', linestyle='', marker='o', markersize=4,
                    zorder=2
                )
            
            else:
            
                # AV18
                if nucleus_name != 'C12':
                    axs[i].errorbar(
                        q_vmc_2b, n_vmc_2b, yerr=n_errors_2b, color='tab:red',
                        label='VMC 2b', linestyle='', marker='s', markersize=4,
                        zorder=1
                    )
                # AV18 + UIX
                axs[i].errorbar(
                    q_vmc_3b, n_vmc_3b, yerr=n_errors_3b, color='tab:blue',
                    label='VMC 2b+3b', linestyle='', marker='o', markersize=4,
                    zorder=2
                )

        # Load data from file
        q_array, q_weights, n_array, n_errors, n_ipm_array, _, _, _, _ = (
            load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb)
        )
    
        # Multiply by factor (2\pi)^3 / Z (N) to compare to VMC data
        n_array *= (2*np.pi) ** 3 / factor
        n_errors *= (2*np.pi) ** 3 / factor
        n_ipm_array *= (2*np.pi) ** 3 / factor

        # Add to plot
        if y_scale == 'linear':
        
            # SRG
            axs[i].plot(q_array, n_array * q_array ** 2, color='black',
                        label='SRG', linewidth=2.0, zorder=4)
            # IPM
            axs[i].plot(
                q_array, n_ipm_array * q_array ** 2, color='tab:orange',
                label='IPM', linestyle='dotted', linewidth=2.0, zorder=3
            )

        else:
        
            # SRG
            axs[i].plot(q_array, n_array, color='black', label='SRG',
                        linewidth=2.0, zorder=4)
            # IPM
            axs[i].plot(q_array, n_ipm_array, color='tab:orange', label='IPM',
                        linestyle='dotted', linewidth=2.0, zorder=3)

        # Set axes limits
        axs[i].set_xlim(x_limits)
        axs[i].set_ylim(y_limits)

        # Set axes labels
        axs[i].set_xlabel(r"$q$ [fm" + r'$^{-1}$' + ']', fontsize=16)
        if i == 0:  # First column only
            if y_scale == 'linear' and nucleon == 'proton':
                ylabel = r"$q^2 n_{p}(q)/Z$ [fm]"
            elif y_scale == 'linear' and nucleon == 'neutron':
                ylabel = r"$q^2 n_{n}(q)/N$ [fm]"
            elif y_scale == 'log' and nucleon == 'proton':
                ylabel = r"$n_{p}(q)/Z$ [fm" + r'$^3$' + ']'
            elif y_scale == 'log' and nucleon == 'neutron':
                ylabel = r"$n_{n}(q)/N$ [fm" + r'$^3$' + ']'
            axs[i].set_ylabel(ylabel, fontsize=16)

        # Add nucleus label
        if y_scale == 'log':
            nucleus_label_loc = 'lower left'
        elif y_scale == 'linear':
            nucleus_label_loc = 'upper left'
        axs[i].add_artist(
            AnchoredText(label_nucleus(nucleus_name), loc=nucleus_label_loc,
                         prop=dict(size=18), frameon=False)
        )
        
    # Set legend in first panel only
    axs[0].legend(loc='upper right', fontsize=13, frameon=False)
    
    # Add potential and \lambda label in last panel
    axs[-1].add_artist(
        AnchoredText(label_potential_lambda(kvnn, lamb), loc='upper right',
                     prop=dict(size=18), frameon=False)
    )

    # Amount of white space in-between sub-plots
    f.subplots_adjust(wspace=0.1)

    # Save?
    if save:   
        file_name = replace_periods(
            f"{nucleon}_momentum_distributions_kvnn_{kvnn}_lamb_{lamb}"
            f"_{y_scale}_scale"
        )
        f.savefig(figures_directory + file_name + '.png')

In [ ]:
# \lambda = 1.5 fm^-1 log y scale

nuclei = (('He4', 2, 2), ('C12', 6, 6), ('O16', 8, 8), ('Ca40', 20, 20))

single_nucleon_momentum_distributions_multipanel(
    nuclei, 'proton', 6, 1.5, y_limits=(1e-4, 5e2), show_vmc=True, save=True
)

In [ ]:
# \lambda = 1.5 fm^-1 linear y scale

nuclei = (('He4', 2, 2), ('C12', 6, 6), ('O16', 8, 8), ('Ca40', 20, 20))

single_nucleon_momentum_distributions_multipanel(
    nuclei, 'proton', 6, 1.5, x_limits=(0.0, 2.0), y_limits=(-0.1, 30),
    y_scale='linear', show_vmc=True, save=True
)

## Contributions of single-nucleon momentum distribution

In [ ]:
def single_nucleon_momentum_distribution_contributions(
        nucleus_name, nucleon, kvnn, lamb, number_of_curves=4,
        x_limits=(0.0,6.0), y_limits=(9e-6,1e3), lda=False,
        print_normalization=False, save=False
):
    """Plots momentum distributions showing I, \delta U + \delta U^\dagger, and
    \delta U \delta U^\dagger contributions.
    """
    
    # Initialize figure
    plt.close('all')
    f, ax = plt.subplots(figsize=(4, 4))

    # Load data from file
    if lda:  # LDA version
        
        md = MomentumDistribution(kvnn, 15.0, 3.0, 120)
        n_total_func, n_I_func, n_delU_func, n_delU2_func = (
            md.get_single_nucleon_momentum_distribution(
                nucleon, nucleus_name, 'SLy4', ('1S0', '3S1'), 'Wegner', lamb,
                contributions=True, interpolate=True
            )
        )
        
        q_array, q_weights = momentum_mesh(15.0, 3.0, 120)
        n_array = n_total_func(q_array)
        n_I_array = n_I_func(q_array)
        n_delU_array = n_delU_func(q_array)
        n_delU2_array = n_delU2_func(q_array)
        
    else:  # Woods-Saxon version
        
        (q_array, q_weights, n_array, _, n_I_array, n_delU_array, _,
         n_delU2_array, _) = load_momentum_distribution(nucleus_name, nucleon,
                                                        kvnn, lamb)
    
    # Print normalization?
    if print_normalization:

        total_norm = compute_normalization(q_array, q_weights, n_array)
        print(f"Total normalization = {total_norm}.")
        
        I_norm = compute_normalization(q_array, q_weights, n_I_array)
        print(f"I normalization = {I_norm}.")
        
        delU_norm = compute_normalization(q_array, q_weights, n_delU_array)
        print(f"\delta U + \delta U^\dagger normalization = {delU_norm}.")
        
        delU2_norm = compute_normalization(q_array, q_weights, n_delU2_array)
        print(f"\delta U \delta U^\dagger normalization = {delU2_norm}.")
        
        relative_error = np.abs(delU_norm + delU2_norm) / delU2_norm * 100
        print(f"Relative error = {relative_error}%.")
        
    # Multiply by factor (2\pi)^3
    if lda:
        factor = 1
    else:
        factor = (2*np.pi) ** 3
    n_array *= factor
    n_I_array *= factor
    n_delU_array *= factor
    n_delU2_array *= factor

    # Add to plot
    ax.semilogy(q_array, n_array, color='black', label='Total', linewidth=2.0,
                zorder=1)
    if number_of_curves > 1:
        ax.semilogy(q_array, n_I_array, color='blue', label='I',
                    linestyle='dotted', linewidth=2.0, zorder=2)
    if number_of_curves > 2:
        ax.semilogy(
            q_array, np.abs(n_delU_array), color='green',
            label=r'$|\delta U+\delta U^\dagger|$', linestyle='dashed',
            linewidth=2.0, zorder=3
        )
    if number_of_curves > 3:
        ax.semilogy(
            q_array, n_delU2_array, color='red',
            label=r'$\delta U \delta U^\dagger$', linestyle='dashdot',
            linewidth=2.0, zorder=4
        )
    
    # Set axes limits
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)

    # Set axes labels
    ax.set_xlabel(r"$q$ [fm" + r'$^{-1}$' + ']', fontsize=16)
    if nucleon == 'proton':
        ylabel = r"$n_{p}(q)$ [fm" + r'$^3$' + ']'
    elif nucleon == 'neutron':
        ylabel = r"$n_{n}(q)$ [fm" + r'$^3$' + ']'
    ax.set_ylabel(ylabel, fontsize=16)

    # Set legend
    ax.legend(loc='upper right', fontsize=14, frameon=False)

    # Add nucleus label
    ax.add_artist(
        AnchoredText(label_nucleus(nucleus_name), loc='lower left',
                     prop=dict(size=18), frameon=False)
    )
    
    # Add potential and \lambda label
    ax.add_artist(
        AnchoredText(label_potential_lambda(kvnn, lamb), loc='center right',
                     prop=dict(size=18), frameon=False)
    )

    # Save?
    if save:
        if number_of_curves >= 4:
            file_name = replace_periods(
                f"{nucleus_name}_{nucleon}_momentum_distribution_contributions"
                f"_kvnn_{kvnn}_lamb_{lamb}"
            )
        else:
            file_name = replace_periods(
                f"{nucleus_name}_{nucleon}_momentum_distribution_contributions"
                f"_kvnn_{kvnn}_lamb_{lamb}_{number_of_curves}"
            )
        if lda:
            file_name += "_lda"
        f.savefig(figures_directory + file_name + ".png")

In [ ]:
# 16O contributions at \lambda = 1.5 fm^-1
single_nucleon_momentum_distribution_contributions(
    'O16', 'proton', 6, 1.5, y_limits=(1e-2, 1e3), print_normalization=True,
    save=True
)

In [ ]:
# 16O contributions at \lambda = 1.5 fm^-1
single_nucleon_momentum_distribution_contributions(
    'O16', 'proton', 6, 1.5, y_limits=(1e-2, 1e3), lda=True, save=True
)

## $\frac{A}{^{4}\rm{He}}$ ratios

In [ ]:
def single_nucleon_momentum_distributions_vary_nucleus(
        nuclei, nucleon, kvnn, lamb, x_limits=(0.0,6.0), y_limits=(0.0,2.0),
        save=False
):
    """Plots momentum distributions varying the NN potential."""
    
    # Initialize figure
    plt.close('all')
    f, ax = plt.subplots(figsize=(4, 4))

    # Get 4He denominator
    (q_array, q_weights, n_4he_array, _, _, _, _, _, _) = (
        load_momentum_distribution('He4', nucleon, kvnn, lamb)
    )
    
    # Loop over potentials
    for i, nucleus in enumerate(nuclei):
        
        nucleus_name, Z, N = nucleus
        A = Z + N

        # Load data from file
        (q_array, q_weights, n_array, _, _, _, _, _, _) = (
            load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb)
        )
        
        ratio_array = (4 * n_array) / (A * n_4he_array)

        # Add to plot
        ax.plot(q_array, ratio_array, label=label_nucleus(nucleus_name),
                linewidth=2.0)

    # Set axes limits
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)

    # Set axes labels
    ax.set_xlabel(r"$q$ [fm" + r'$^{-1}$' + ']', fontsize=16)
    ax.set_ylabel(r"$A / ^{4}$He ratio", fontsize=16)

    # Set legend
    ax.legend(loc='lower right', fontsize=16, frameon=False)
    
    # Add potential and \lambda label
    ax.add_artist(
        AnchoredText(label_potential_lambda(kvnn, lamb), loc='upper left',
                     prop=dict(size=18), frameon=False)
    )
    
    # Save?
    if save:
        file_name = (f"{nucleon}_momentum_distributions_ratios_kvnn_{kvnn}"
                     f"_lamb_{lamb}")
        for nucleus in nuclei:
            nucleus_name, _, _ = nucleus
            file_name += f"_{nucleus_name}"
        file_name = replace_periods(file_name)
        f.savefig(figures_directory + file_name + ".png")

In [ ]:
# nuclei = (('C12', 6, 6), ('O16', 8, 8), ('Ca40', 20, 20))

# single_nucleon_momentum_distributions_vary_nucleus(
#     nuclei, 'proton', 6, 1.5, x_limits=(2.0, 6.0), y_limits=(0.0,2.0),
#     save=True
# )

## Vary potential

In [ ]:
def single_nucleon_momentum_distributions_vary_kvnn(
        nucleus_name, nucleon, kvnns, lamb, x_limits=(0.0,6.0),
        y_limits=(9e-6,1e3), y_scale='log', legend_fontsize=11, save=False
):
    """Plots momentum distributions varying the NN potential."""
    
    # Initialize figure
    plt.close('all')
    f, ax = plt.subplots(figsize=(4, 4))
    
    if y_scale == 'log':
        ax.set_yscale('log')
    else:
        ax.set_yscale('linear')
    
    # Loop over potentials
    for i, kvnn in enumerate(kvnns):
        
        # Load data from file
        (q_array, q_weights, n_array, _, _, _, _, _, _) = (
            load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb)
        )
    
        # Multiply by factor (2\pi)^3
        n_array *= (2*np.pi) ** 3

        # Add to plot
        if y_scale == 'linear':  # Linear scale
            ax.plot(q_array, n_array * q_array ** 2, label=label_kvnn(kvnn),
                    ls=line_styles(i), lw=2.0, zorder=i)
        elif y_scale == 'log':  # Log scale
            ax.plot(q_array, n_array, label=label_kvnn(kvnn), ls=line_styles(i),
                    lw=2.0, zorder=i)

    # Set axes limits
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)

    # Set axes labels
    ax.set_xlabel(r"$q$ [fm" + r'$^{-1}$' + ']', fontsize=16)
    if y_scale == 'linear' and nucleon == 'proton':
        ylabel = r"$q^2 n_{p}(q)$ [fm]"
    elif y_scale == 'linear' and nucleon == 'neutron':
        ylabel = r"$q^2 n_{n}(q)$ [fm]"
    elif y_scale == 'log' and nucleon == 'proton':
        ylabel = r"$n_{p}(q)$ [fm" + r'$^3$' + ']'
    elif y_scale == 'log' and nucleon == 'neutron':
        ylabel = r"$n_{n}(q)$ [fm" + r'$^3$' + ']'
    ax.set_ylabel(ylabel, fontsize=16)

    # Set legend
    ax.legend(loc='upper right', fontsize=legend_fontsize, frameon=False)

    # Add nucleus label
    ax.add_artist(
        AnchoredText(label_nucleus(nucleus_name), loc='center right',
                     prop=dict(size=18), frameon=False)
    )
    
    # Add \lambda label
    ax.add_artist(
        AnchoredText(label_lambda(lamb), loc='lower left', prop=dict(size=18),
                     frameon=False)
    )
    
    # Save?
    if save:
        file_name = f"{nucleus_name}_{nucleon}_momentum_distributions_kvnns"
        for kvnn in kvnns:
            file_name += f"_{kvnn}"
        file_name += f"_lamb_{lamb}_{y_scale}_scale"
        file_name = replace_periods(file_name)
        f.savefig(figures_directory + file_name + ".png")

In [ ]:
single_nucleon_momentum_distributions_vary_kvnn(
    'O16', 'proton', (6, 7, 113, 79), 1.5, y_limits=(1e-2,1e3), save=True
)

## Vary $\lambda$

In [ ]:
def single_nucleon_momentum_distributions_vary_lambda(
        nucleus_name, nucleon, kvnn, lamb, lamb_min=1.35, lamb_max=2.0,
        x_limits=(0.0,6.0), y_limits=(9e-6,1e3), y_scale='log',
        nucleus_label=False, kvnn_label=False, legend=False, save=False
):
    """Plots momentum distributions varying the SRG flow parameter \lambda."""
    
    # Initialize figure
    plt.close('all')
    f, ax = plt.subplots(figsize=(4, 4))
    
    if y_scale == 'log':
        ax.set_yscale('log')
    else:
        ax.set_yscale('linear')
        
    # Create light red band from minimum \lambda to maximum \lambda
    (q_lmin_array, _, n_lmin_array, _, _, _, _, _, _) = (
        load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb_min)
    )
    n_lmin_array *= (2*np.pi) ** 3  # Same scale as VMC
    
    (q_lmax_array, _, n_lmax_array, _, _, _, _, _, _) = (
        load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb_max)
    )
    n_lmax_array *= (2*np.pi) ** 3  # Same scale as VMC
    
    # Input \lambda and IPM
    (q_array, q_weights, n_array, _, n_ipm_array, _, _, _, _) = (
        load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb)
    )
    n_array *= (2*np.pi) ** 3  # Same scale as VMC
    n_ipm_array *= (2*np.pi) ** 3
    
    # Add curves to plot
    if y_scale == 'linear':
        
        # Light red band
        ax.fill_between(
            q_lmin_array, y1=n_lmax_array * q_lmin_array ** 2,
            y2=n_lmin_array * q_lmin_array ** 2, color='red', alpha=0.15
        )
        # IPM
        ax.plot(q_array, n_ipm_array * q_array ** 2, color='black',
                label=label_lambda(np.inf), ls='dotted', lw=2.0)
        # SRG
        ax.plot(q_array, n_array * q_array ** 2, color='red',
                label=label_lambda(lamb), lw=2.0)
        
    elif y_scale == 'log':
        
        # Light red band
        ax.fill_between(q_lmin_array, y1=n_lmax_array, y2=n_lmin_array,
                        color='red', alpha=0.15)
        # IPM
        ax.plot(q_array, n_ipm_array, color='black', label=label_lambda(np.inf),
                ls='dotted', lw=2.0)
        # SRG
        ax.plot(q_array, n_array, color='red', label=label_lambda(lamb), lw=2.0)

    # Set axes limits
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)

    # Set axes labels
    ax.set_xlabel(r"$q$ [fm" + r'$^{-1}$' + ']', fontsize=16)
    if y_scale == 'linear' and nucleon == 'proton':
        ylabel = r"$q^2 n_{p}(q)$ [fm]"
    elif y_scale == 'linear' and nucleon == 'neutron':
        ylabel = r"$q^2 n_{n}(q)$ [fm]"
    elif y_scale == 'log' and nucleon == 'proton':
        ylabel = r"$n_{p}(q)$ [fm" + r'$^3$' + ']'
    elif y_scale == 'log' and nucleon == 'neutron':
        ylabel = r"$n_{n}(q)$ [fm" + r'$^3$' + ']'
    ax.set_ylabel(ylabel, fontsize=16)

    # Set legend
    if legend:
        ax.legend(loc='upper right', fontsize=16, frameon=False)

    # Add nucleus label
    if nucleus_label:
        ax.add_artist(
            AnchoredText(label_nucleus(nucleus_name), loc='lower left',
                         prop=dict(size=18), frameon=False)
        )
        
    # Add potential label
    if kvnn_label:
        ax.add_artist(
            AnchoredText(label_kvnn(kvnn), loc='upper right',
                         prop=dict(size=18), frameon=False)
        )
    
    # Save?
    if save:
        file_name = (f"{nucleus_name}_{nucleon}_momentum_distributions_kvnn"
                     f"_{kvnn}_vary_lambda_{lamb}_{y_scale}_scale")
        file_name = replace_periods(file_name)
        f.savefig(figures_directory + file_name + ".png")

In [ ]:
single_nucleon_momentum_distributions_vary_lambda(
    'O16', 'proton', 6, 1.5, y_limits=(1e-3, 1e3), nucleus_label=True,
    legend=True, save=True
)

In [ ]:
single_nucleon_momentum_distributions_vary_lambda(
    'O16', 'proton', 6, 1.5, x_limits=(0.0, 2.0), y_limits=(-0.1, 210.0),
    y_scale='linear', kvnn_label=True, save=True
)

## Matching interactions

In [ ]:
def matching_distribution(
        nucleus_name, nucleon, kvnn, lamb, kvnn_hard, lambm, lambm_min,
        lambm_max, x_limits=(0.0,6.0), y_limits=(9e-6,1e3), y_scale='log',
        nucleus_label=False, lamb_label=False, legend=False, save=False
):
    """Plots momentum distributions matching a soft and hard potential."""
    
    # Initialize figure
    plt.close('all')
    f, ax = plt.subplots(figsize=(4, 4))
    
    if y_scale == 'log':
        ax.set_yscale('log')
    else:
        ax.set_yscale('linear')
        
    # Hard potential
    (q_array, q_weights, n_hard_array, _, _, _, _, _, _) = (
        load_momentum_distribution(nucleus_name, nucleon, kvnn_hard, lamb)
    )
    n_hard_array *= (2*np.pi) ** 3  # Same scale as VMC
        
    # Soft potential
    (q_array, q_weights, n_soft_array, _, _, _, _, _, _) = (
        load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb)
    )
    n_soft_array *= (2*np.pi) ** 3  # Same scale as VMC
    
    # Soft potential with initial 2-body operator
    (q_array, q_weights, n_lambm_array, _, _, _, _, _, _) = (
        load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb, kvnn_hard,
                                   lambm)
    )
    n_lambm_array *= (2*np.pi) ** 3  # Same scale as VMC
        
    # Create light red band from minimum \lambda_m to maximum \lambda_m
    (q_array, _, n_lmin_array, _, _, _, _, _, _) = (
        load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb, kvnn_hard,
                                   lambm_min)
    )
    n_lmin_array *= (2*np.pi) ** 3  # Same scale as VMC
    
    (q_array, _, n_lmax_array, _, _, _, _, _, _) = (
        load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb, kvnn_hard,
                                   lambm_max)
    )
    n_lmax_array *= (2*np.pi) ** 3  # Same scale as VMC
    
    # Label for \lambda_m
    lambm_label = rf"$\lambda_m={lambm}$" + " fm" + r"$^{-1}$"
    
    # Add curves to plot
    if y_scale == 'linear':
        
        # Light red band
        ax.fill_between(
            q_array, y1=n_lmax_array * q_array ** 2,
            y2=n_lmin_array * q_array ** 2, color='red', alpha=0.15, zorder=3
        )
        # Soft potential
        ax.plot(q_array, n_soft_array * q_array ** 2, color='black',
                label=label_kvnn(kvnn), ls='dotted', lw=2.0, zorder=1)
        # Hard potential
        ax.plot(q_array, n_hard_array * q_array ** 2, color='blue',
                label=label_kvnn(kvnn_hard), ls='solid', lw=2.0, zorder=2)
        # Matched distribution
        ax.plot(q_array, n_lambm_array * q_array ** 2, color='red',
                label=lambm_label, ls='dashed', lw=2.0, zorder=4)
        
    elif y_scale == 'log':
        
        # Light red band
        ax.fill_between(q_array, y1=n_lmax_array, y2=n_lmin_array, color='red',
                        alpha=0.15, zorder=3)
        # Soft potential
        ax.plot(q_array, n_soft_array, color='black', label=label_kvnn(kvnn),
                ls='dotted', lw=2.0, zorder=1)
        # Hard potential
        ax.plot(q_array, n_hard_array, color='blue',
                label=label_kvnn(kvnn_hard), ls='solid', lw=2.0, zorder=2)
        # Matched distribution
        ax.plot(q_array, n_lambm_array, color='red', label=lambm_label,
                ls='dashed', lw=2.0, zorder=4)

    # Set axes limits
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)

    # Set axes labels
    ax.set_xlabel(r"$q$ [fm" + r'$^{-1}$' + ']', fontsize=16)
    if y_scale == 'linear' and nucleon == 'proton':
        ylabel = r"$q^2 n_{p}(q)$ [fm]"
    elif y_scale == 'linear' and nucleon == 'neutron':
        ylabel = r"$q^2 n_{n}(q)$ [fm]"
    elif y_scale == 'log' and nucleon == 'proton':
        ylabel = r"$n_{p}(q)$ [fm" + r'$^3$' + ']'
    elif y_scale == 'log' and nucleon == 'neutron':
        ylabel = r"$n_{n}(q)$ [fm" + r'$^3$' + ']'
    ax.set_ylabel(ylabel, fontsize=16)

    # Set legend
    if legend:
        ax.legend(loc='upper right', fontsize=13, frameon=False)

    # Add nucleus label
    if nucleus_label:
        ax.add_artist(
            AnchoredText(label_nucleus(nucleus_name), loc='center left',
                         prop=dict(size=18), frameon=False)
        )
        
    # Add \lambda label
    ax.add_artist(
        AnchoredText(label_lambda(lamb), loc='lower left', prop=dict(size=18),
                     frameon=False)
    )
    
    # Save?
    if save:
        file_name = (
            f"{nucleus_name}_{nucleon}_matching_distribution_kvnn_{kvnn}_lamb"
            f"_{lamb}_kvnn_hard_{kvnn_hard}_lambm_{lambm}_{y_scale}_scale"
        )
        file_name = replace_periods(file_name)
        f.savefig(figures_directory + file_name + ".png")

In [ ]:
# Matching SMS N4LO 550 MeV to AV18
matching_distribution(
    'O16', 'proton', 113, 1.5, 6, 4.5, 4.0, 5.0, y_limits=(1e-3, 1e3),
    nucleus_label=True, lamb_label=True, legend=True, save=True
)

### _Testing difference between AV18 and AV18+UX_

In [ ]:
def test_3body(nucleus_name, x_limits=(0.0,6.0), y_limits=(1e-4,1e3),
               y_scale='log'):
    """Testing He4 with and without UX force."""
    
    filename_2body_only = f"{nucleus_name}_single_nucleon_av18.txt"
    data_2body_only = np.loadtxt(vmc_directory + filename_2body_only)
    
    filename_3body = f"{nucleus_name}_single_nucleon_av18_uix.txt"
    data_3body = np.loadtxt(vmc_directory + filename_3body)
    
    q_2body_only = data_2body_only[:,0]
    n_2body_only = data_2body_only[:,1]
    n_errors_2body_only = data_2body_only[:,2]
    
    q_3body = data_3body[:,0]
    n_3body = data_3body[:,1]
    n_errors_3body = data_3body[:,2]
    
    plt.close('all')
    f, ax = plt.subplots(figsize=(4, 4))
        
    if y_scale == 'linear':
        
        ax.set_yscale('linear')
        ax.errorbar(
            q_2body_only, n_2body_only * q_2body_only ** 2,
            yerr=n_errors_2body_only, label='AV18', linestyle='', marker='.'
        )
        ax.errorbar(
            q_3body, n_3body * q_3body ** 2, yerr=n_errors_3body,
            label='AV18+UIX', linestyle='', marker='.'
        )
        
    elif y_scale == 'log':
        
        ax.set_yscale('log')
        ax.errorbar(
            q_2body_only, n_2body_only, yerr=n_errors_2body_only, label='AV18',
            linestyle='', marker='.'
        )
        ax.errorbar(q_3body, n_3body, yerr=n_errors_3body, label='AV18+UIX',
                    linestyle='', marker='.')
        
    # Load data from file
    q_array, q_weights, n_array, n_errors, n_ipm_array, _, _, _, _ = (
        load_momentum_distribution(nucleus_name, 'proton', 6, 1.5)
    )
    
    # Multiply by factor (2\pi)^3 / Z (N) to compare to VMC data
    n_array *= (2*np.pi) ** 3
    n_errors *= (2*np.pi) ** 3
    n_ipm_array *= (2*np.pi) ** 3

    # Add to plot
    if y_scale == 'linear':
        
        ax.plot(q_array, n_array * q_array ** 2, color='black', label='SRG',
                lw=2.0)

    else:
        
        ax.plot(q_array, n_array, color='black', label='SRG', linewidth=2.0)
        
    # Set axes limits
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)

    # Set axes labels
    ax.set_xlabel(r"$q$ [fm" + r'$^{-1}$' + ']', fontsize=16)
    if y_scale == 'linear':
        ylabel = r"$q^2 n(q)$ [fm]"
    elif y_scale == 'log':
        ylabel = r"$n(q)/$ [fm" + r'$^3$' + ']'
    ax.set_ylabel(ylabel, fontsize=16)

    # Add nucleus label
    if y_scale == 'log':
        nucleus_label_loc = 'lower left'
    elif y_scale == 'linear':
        nucleus_label_loc = 'upper left'
    ax.add_artist(
        AnchoredText(label_nucleus(nucleus_name), loc=nucleus_label_loc,
                     prop=dict(size=18), frameon=False)
    )
        
    # Set legend in first panel only
    ax.legend(loc='upper right', fontsize=16, frameon=False);

In [ ]:
test_3body('He4', y_limits=(1e-3,1e3))

In [ ]:
test_3body('He4', x_limits=(0,2), y_limits=(0,60), y_scale='linear')

In [ ]:
# He4

T_2body_only = 96.801  # Kinetic energy with AV18
T_3body = 110.7  # Kinetic energy with AV18 + UIX
relative_error = np.abs(T_3body - T_2body_only) / T_3body * 100
    
print(f"AV18 <T> = {T_2body_only:.3f} MeV")
print(f"AV18+UX <T> = {T_3body:.3f} MeV")
print(f"Relative error = {relative_error:.2f} percent")

In [ ]:
test_3body('O16', y_limits=(1e-3,1e3))

In [ ]:
test_3body('O16', x_limits=(0,2), y_limits=(0,250), y_scale='linear')

In [ ]:
# O16

T_2body_only = 504.0122  # Kinetic energy with AV18
T_3body = 461.33  # Kinetic energy with AV18 + UIX
relative_error = np.abs(T_3body - T_2body_only) / T_3body * 100
    
print(f"AV18 <T> = {T_2body_only:.3f} MeV")
print(f"AV18+UIX <T> = {T_3body:.3f} MeV")
print(f"Relative error = {relative_error:.2f} percent")

### _Test different Woods-Saxon_

In [ ]:
def load_momentum_distribution_test(
    nucleus_name, nucleon, kvnn, lamb, kvnn_hard=None, lambda_m=None
):
    """Load and return the momentum distribution along with the isolated
    contributions.
    """
    
    directory = f"../data/momentum_distributions/{nucleus_name}/"

    if kvnn_hard is not None:
        file_name = replace_periods(
            f"{nucleus_name}_{nucleon}_momentum_distribution_kvnn_{kvnn}_lamb"
            f"_{lamb}_kvnn_hard_{kvnn_hard}_lambda_m_{lambda_m}"
        )
    else:
        file_name = replace_periods(f"{nucleus_name}_{nucleon}_momentum"
                                    f"_distribution_kvnn_{kvnn}_lamb_{lamb}")
        
    file_name += "_test"
    
    data = np.loadtxt(directory + file_name + '.txt')
    
    q_array = data[:, 0]
    q_weights = data[:, 1]
    n_array = data[:, 2]
    n_errors = data[:, 3]
    I_array = data[:, 4]
    delta_U_array = data[:, 5]
    delta_U_errors = data[:, 6]
    delta_U2_array = data[:, 7]
    delta_U2_errors = data[:, 8]
    
    return (q_array, q_weights, n_array, n_errors, I_array, delta_U_array,
            delta_U_errors, delta_U2_array, delta_U2_errors)

In [ ]:
def compare_ws(
        nucleus_name, x_limits=(0.0,6.0), y_limits=(1e-4,1e3), y_scale='log'
):
    """Testing C12 with different Woods-Saxon parametrizations."""
    
    # nucleus_name = 'C12'
    
    
    filename_uix = f"{nucleus_name}_single_nucleon_av18_uix.txt"
    data_uix = np.loadtxt(vmc_directory + filename_uix)
    
    if nucleus_name == 'C12':
        filename_ux = f"{nucleus_name}_single_nucleon_av18_ux.txt"
        data_ux = np.loadtxt(vmc_directory + filename_ux)
    elif nucleus_name == 'O16':
        filename_ux = f"{nucleus_name}_single_nucleon_av18.txt"
        data_ux = np.loadtxt(vmc_directory + filename_ux)

    q_uix = data_uix[:,0]
    n_uix = data_uix[:,1]
    n_errors_uix = data_uix[:,2]
    
    q_ux = data_ux[:,0]
    n_ux = data_ux[:,1]
    n_errors_ux = data_ux[:,2]
    
    plt.close('all')
    f, ax = plt.subplots(figsize=(4, 4))
        
    if y_scale == 'linear':
        
        ax.set_yscale('linear')
        ax.errorbar(q_uix, n_uix * q_uix ** 2, yerr=n_errors_uix,
                    label='AV18+UIX', ls='', marker='.')
        if nucleus_name == 'C12':
            ax.errorbar(q_ux, n_ux * q_ux ** 2, yerr=n_errors_ux,
                        label='AV18+UX', ls='', marker='.')
        elif nucleus_name == 'O16':
            ax.errorbar(q_ux, n_ux * q_ux ** 2, yerr=n_errors_ux,
                        label='AV18', ls='', marker='.')
        
    elif y_scale == 'log':
        
        ax.set_yscale('log')
        ax.errorbar(q_uix, n_uix, yerr=n_errors_uix, label='AV18+UIX', ls='',
                    marker='.')
        if nucleus_name == 'C12':
            ax.errorbar(q_ux, n_ux, yerr=n_errors_ux, label='AV18+UX', ls='',
                        marker='.')
        elif nucleus_name == 'O16':
            ax.errorbar(q_ux, n_ux, yerr=n_errors_ux, label='AV18', ls='',
                        marker='.')
        
    # Load data from file
    q_array, q_weights, n_array, n_errors, n_ipm_array, _, _, _, _ = (
        load_momentum_distribution(nucleus_name, 'proton', 6, 1.5)
    )
    (q_new_array, q_new_weights, n_new_array, n_new_errors, n_new_ipm_array, _,
    _, _, _) = (
        load_momentum_distribution_test(nucleus_name, 'proton', 6, 1.5)
    )
    
    # Multiply by factor (2\pi)^3 / Z (N) to compare to VMC data
    n_array *= (2*np.pi) ** 3
    n_errors *= (2*np.pi) ** 3
    n_ipm_array *= (2*np.pi) ** 3
    n_new_array *= (2*np.pi) ** 3
    n_new_errors *= (2*np.pi) ** 3
    n_new_ipm_array *= (2*np.pi) ** 3

    # Add to plot
    if y_scale == 'linear':
        
        ax.plot(q_array, n_array * q_array ** 2, color='black', label='Univ.',
                lw=2.0)
        ax.plot(q_new_array, n_new_array * q_new_array ** 2, color='tab:red',
                label='New WS', ls='dashdot', lw=2.0)

    else:
        
        ax.plot(q_array, n_array, color='black', label='Univ.', linewidth=2.0)
        ax.plot(q_new_array, n_new_array, color='tab:red', label='New WS',
                ls='dashdot', lw=2.0)
        
    # Set axes limits
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)

    # Set axes labels
    ax.set_xlabel(r"$q$ [fm" + r'$^{-1}$' + ']', fontsize=16)
    if y_scale == 'linear':
        ylabel = r"$q^2 n(q)$ [fm]"
    elif y_scale == 'log':
        ylabel = r"$n(q)$ [fm" + r'$^3$' + ']'
    ax.set_ylabel(ylabel, fontsize=16)

    # Add nucleus label
    if y_scale == 'log':
        nucleus_label_loc = 'lower left'
    elif y_scale == 'linear':
        nucleus_label_loc = 'upper left'
    ax.add_artist(
        AnchoredText(label_nucleus(nucleus_name), loc=nucleus_label_loc,
                     prop=dict(size=18), frameon=False)
    )
        
    # Set legend in first panel only
    ax.legend(loc='upper right', fontsize=14, frameon=False);

In [ ]:
compare_ws('C12', x_limits=(0.0,6.0), y_limits=(1e-3,1e3))

In [ ]:
compare_ws('C12', x_limits=(0,2), y_limits=(0,180), y_scale='linear')

In [ ]:
compare_ws('O16', x_limits=(0.0,6.0), y_limits=(1e-3,1e3))

In [ ]:
compare_ws('O16', x_limits=(0,2), y_limits=(0,230), y_scale='linear')

### _Testing radii calculations_

In [6]:
from test_momentum_distribution_script_v9 import WoodsSaxon, compute_clebsch_gordan_table

In [7]:
def rms_radius(nucleus_name, Z, N, test=False):
    """Compute expectation value of radius."""
    
    jmax = 3
    cg_table = compute_clebsch_gordan_table(jmax)
    woods_saxon = WoodsSaxon(nucleus_name, Z, N, cg_table, test=test)
    dr = woods_saxon.dr
    
    r2_proton, r2_neutron = 0, 0
    for sp_state in woods_saxon.occupied_states:
    
        # u^p(r)
        r_array, u_array = woods_saxon.get_wf_rspace(sp_state)
        
        if sp_state.m_t == 1/2:  # Proton
            r2_proton += np.sum(dr * r_array ** 2 * np.abs(u_array) ** 2)
        elif sp_state.m_t == -1/2:  # Neutron
            r2_neutron += np.sum(dr * r_array ** 2 * np.abs(u_array) ** 2)

    r_proton = np.sqrt(r2_proton / Z)
    r_neutron = np.sqrt(r2_neutron / N)
    
    return r_proton, r_neutron

In [8]:
# # Compute Woods-Saxon RMS radii here
# r_rms_univ, _ = rms_radius('C12', 6, 6)
# r_rms_test, _ = rms_radius('C12', 6, 6, test=True)

In [9]:
# print(f"VMC w/ AV18 + UIX: <r_p> = {2.3484} fm")
# print(f"VMC w/ AV18 + UX: <r_p> = {2.3522} fm")
# print(f"SRG w/ Univ.: <r_p> = {r_rms_univ:.5f} fm")
# print(f"SRG w/ test Woods-Saxon: <r_p> = {r_rms_test:.5f} fm")

In [10]:
# Compute Woods-Saxon RMS radii here
r_rms_univ, _ = rms_radius('O16', 8, 8)
r_rms_test, _ = rms_radius('O16', 8, 8, test=True)

Done calculating Clebsch-Gordan table up to j_max = 3.
Done calculating Clebsch-Gordan table up to j_max = 3.


In [11]:
print(f"SRG w/ Univ.: <r_p> = {r_rms_univ:.5f} fm")
print(f"SRG w/ test Woods-Saxon: <r_p> = {r_rms_test:.5f} fm")

SRG w/ Univ.: <r_p> = 2.59844 fm
SRG w/ test Woods-Saxon: <r_p> = 2.62065 fm


In [12]:
# r_rms_univ, _ = rms_radius('Ca40', 20, 20)
# print(f"SRG w/ Univ.: <r_p> = {r_rms_univ:.5f} fm")